In [3]:
from tqdm import tqdm
from collections import Counter

problem with memory limit (this is a bad notebook, because of bad data)

In [8]:
t = 0 
s = None

def main():
    with open('SCC.txt') as file:
        a = file.read().splitlines()
        a = list(map(lambda x: x.strip().split(" "), a))
        
    D = {}
    D_r = {}
    for x in a:
        v0, v1 = int(x[0]), int(x[1])
        if v0 not in D:
            D[v0] = [v1]
        else:
            D[v0].append(v1)

        if v1 not in D_r:
            D_r[v1] = [v0]
        else:
            D_r[v1].append(v0)
            
    for i in range(1, 875714+1):
        if i not in D:
            D[i] = []
        if i not in D_r:
            D_r[i] = []
        
    n = 875714
    m = len(a)
    
    
    t = 0 
    s = None
    
    D_explored = set()
    Leader = dict()
    Finish_time = dict()


    def DFS(D, i):
        global t, s
        D_explored.add(i)
        Leader[i] = s
        for j in D_r[i]:
            if j not in D_explored:
                DFS(D, j)
        t += 1
        Finish_time[i] = t


    for i in tqdm(range(n,0, -1)):
        if i not in D_explored:
            s = i
            DFS(D, i)


In [ ]:
import sys
sys.setrecursionlimit(300000)
source='SCC.txt'
N=875714
def getG():
    G={}
    Grev={}
    for i in range(1,N+1):
        G[i]=[]
        Grev[i]=[]
    fin=open(source)
    for line in fin:
        v1=int(line.split()[0])
        v2=int(line.split()[1])
        G[v1].append(v2)
        Grev[v2].append(v1)
    fin.close()
    return G, Grev

#globals
visited={}
finish={}
leader={}

def init():
    for i in range(1,N+1):
        visited[i]=0
        finish[i]=0
        leader[i]=0

def dfs(G, i):
    global t
    visited[i]=1
    leader[i]=s
    for j in G[i]:
        if(visited[j]==0): dfs(G,j)
    t=t+1
    finish[i]=t

def dfs_loop(G):
    global t
    global s
    t=0 #number of nodes processed so far
    s=0 #current source vertex
    i=N
    while(i>0):
        if(visited[i]==0):
            s=i
            dfs(G,i)
        i=i-1

if __name__ == "__main__":
    init()
    g, grev=getG()
    dfs_loop(grev) #THE FIRST LOOP ON REVERSE GRAPH

    # construct new graph
    newGraph={}
    for i in range(1,N+1):
        temp=[]
        for x in g[i]: temp.append(finish[x])
        newGraph[finish[i]]=temp

    init()    
    dfs_loop(newGraph) #THE SECOND LOOP 

    # statistics
    lst= sorted(leader.values())
    stat=[]
    pre=0
    for i in range(0,N-1):
        if lst[i]!=lst[i+1]:
            stat.append(i+1-pre)
            pre=i+1
    stat.append(N-pre)
    L= sorted(stat)
    L.reverse()
    print(L[0:5])

In [9]:
import sys, threading
sys.setrecursionlimit(4000000)
threading.stack_size(67108864)
thread = threading.Thread(target=main)
thread.start()

In [ ]:
main()

  0%|          | 0/875714 [00:00<?, ?it/s]

In [6]:
n = 875714
m = len(a)
n, m

(875714, 5105043)

  0%|          | 0/875714 [00:00<?, ?it/s]

In [ ]:
C = Counter(Leader.values())

In [ ]:
C.most_common(5)

In [2]:
import time

In [ ]:
%%time

import collections
import sys
import random
 
def return_emptylist():
  return []
 
def return_false():
  return False
 
def ParseGraph(filename):
    #algo4SCC.txt
  """Parse a graph into a list of edges for programming Q.4
 
Args:
- filename: the on-disk graph representation
Returns:
- edges = [(vertex_1, vertex_2), ...]
"""
  edges = []
  for l in open(filename):
    fields = [int(f) for f in l.split()]
    edges.append(tuple(fields))
 
  adjacency = collections.defaultdict(return_emptylist)
  reverse_adjacency = collections.defaultdict(return_emptylist)
  for e in edges:
    adjacency[e[0]] = adjacency[e[0]] + [e]
    reverse_adjacency[e[1]] = reverse_adjacency[e[1]] + [(e[1], e[0])]
 
  return adjacency, reverse_adjacency, edges
 
t = 0
s = 0
finishing = {}
leader = {}
explored = collections.defaultdict(return_false)
def ResetState():
  global t, s, finishing, leader, explored
  t = 0
  s = 0
  finishing = {}
  leader = {}
  explored = collections.defaultdict(return_false)
 
def DFSLoop(edges, labeling, reversed = False):
  global s
  for i in labeling:
    if not explored[i]:
      s = i
      DFS(edges, i, reversed)
 
forward_adjacency = {}
reverse_adjacency = {}
def DFS(edges, start, reversed = False):
  global t
  if reversed:
    adjacency = reverse_adjacency
  else:
    adjacency = forward_adjacency
 
  # Iterative (i.e. manually managing a stack) solution.
  stack = []
  stack.append((start, 1))
 
  while len(stack) > 0:
    current, phase = stack.pop()
    if phase == 1:
      explored[current] = True
      leader[current] = s
      edge_found = False
      for edge in adjacency[current]:
        if not explored[edge[1]]:
          stack.append((current, 1))
          stack.append((edge[1], 1))
          edge_found = True
          break
      if not edge_found:
        stack.append((current, 2))
    if phase == 2:
      t += 1
      finishing[current] = t
 
forward_adjacency, reverse_adjacency, edges = ParseGraph("SCC.txt")
 
sys.stderr.write('Graph parsed\n')
 
num_nodes = max([e[0] for e in edges] + [e[1] for e in edges])
labeling = range(num_nodes, 0, -1)
DFSLoop(edges, labeling, True)
 
sys.stderr.write('Reverse DFSLoop done\n')
 
inverse_finishing = dict((v, k) for k, v in finishing.items())
finish_labeling = [inverse_finishing[i] for i in range(num_nodes, 0, -1)]
 
ResetState()
DFSLoop(edges, finish_labeling)
 
sys.stderr.write('Forward DFSLoop done\n')
 
sccs = {}
for i in leader:
  if leader[i] not in sccs:
    sccs[leader[i]] = [i]
  else:
    sccs[leader[i]].append(i)
 


In [7]:
from collection import Counter 

371762

In [8]:
sccs[4]

[4]